In [27]:
# STEP ONE set up script
# imports
import os
import time

from arctic import Arctic
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
import alpaca_trade_api as tradeapi

# keys
API_KEY = os.path.expandvars('$ALPACA_PAPER_KEY')
API_SECRET = os.path.expandvars('$ALPACA_SECRET_PAPER_KEY')
ALPCA_API_BASE_URL = "https://paper-api.alpaca.markets"
ALPHA_VANTAGE_KEY = os.path.expandvars("$ALPHA_VANTAGE_HIGHER_KEY")

# alpaca, alpha_vantage, arctic
alpaca = tradeapi.REST(API_KEY, API_SECRET, ALPCA_API_BASE_URL, api_version='v2')
ts = TimeSeries(ALPHA_VANTAGE_KEY, output_format='pandas')
ti = TechIndicators(key=ALPHA_VANTAGE_KEY, output_format='pandas')
store = Arctic('localhost')
library = store['HISTORICAL_DATA']
simple_moving_lib = store['SMA']

# STEP TWO
# Download and Update data
aapl_data, meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
# arctic is versioned, so we can just overwrite the data
library.write('AAPL', aapl_data)

aapl_sma, meta_sma = ti.get_sma(symbol='AAPL', interval='daily')
simple_moving_lib.write('AAPL', aapl_sma)


# STEP THREE / FOUR
# Run models & Trade
# SMA vs yesterday SMA
if aapl_sma['SMA'].tail(1)[0] - aapl_sma['SMA'].tail(2)[0] > 0.2:
    alpaca.submit_order('AAPL', 1, 'buy', 'market', 'day')
